# Creating a Customer Service Agent with Client-Side Tools

In this notebook, we'll demonstrate how to create a household assistant chatbot using gpt-3.5 (via the Azure platoform) plus client-side tools. The chatbot will be able to look up information stored locally (i.e., an inventory of the fridge) as well as to use information from a search query to inform the answer.

## Step 1: Set up the environment
First, let's install the required libraries and set up the openai API client.

### Set up LLM via API call
We will call the LLM (gpt-3.5 turbo, in our case) via an API, i.e. an Application programming interface, i.e. a defined interface for two programs (here: the code in the notebook, and the LLM) can interact.

In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os  
from openai import AzureOpenAI  

# technical set-up - don't change anything here!
endpoint = os.getenv("ENDPOINT_URL", "https://cas-dml-llm.openai.azure.com/")  
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-35-turbo")  
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "986IfxLKwN3Paiq4yx1Kn2iTG7FyG2GxFg17qQSyr1KZqGLaizAGJQQJ99BCACI8hq2XJ3w3AAABACOGQfvw")  

# Initialize Azure OpenAI Service client with key-based authentication    
client = AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2024-05-01-preview",
)

In [3]:
# core funtion to interact with the llm over the API:
def get_ai_response(user_query):
    messages = [
        {"role": "system", "content": "You are an AI assistant that helps people find information."},
        {"role": "user", "content": user_query}
    ]

    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
        max_tokens=800,
        temperature=0, #0.7,
        top_p=0.95
    )

    return response.choices[0].message.content  # Extract AI's reply

In [4]:
# Example query:
user_input = "What is the meaning of life according to Douglas Adams, give me only the number?"
answer = get_ai_response(user_input)
print(answer)

42


### Web search
The following allows for web searches - note that most API calls usually don't allow the LLM to search the web.

In [5]:
SERPAPI_API_KEY = "e5ae293efd4fa604f2e5ec848310ff76c41078366082c175fe7e86d7a2aa6d24"
%pip install serpapi
#%pip install pandas

Note: you may need to restart the kernel to use updated packages.


Here we define the function to do the web search.

In [6]:
import os
import serpapi

def get_search_result(query, answer_field='answer_box'):
    client = serpapi.Client(api_key=SERPAPI_API_KEY)
    results = client.search({
        'q':query,
        'engine':"google",
    })

    if answer_field not in results:
        return "No answer found"
    
    return results[answer_field]

### Initialize Local Applications
We will use a fridge inventory as a simple case of local data that should be kept up to date and be a data source for some functionality. For the sake of simplicity, we use a simple data frame and a `csv` file; for a proper application, this should probably be handled as a data base.

In [7]:
import pandas as pd

sample_data = [
    {"Item": "Milk", "Quantity": 2, "Expiry Date": "2025-03-10"},
    {"Item": "Eggs", "Quantity": 12, "Expiry Date": "2025-03-15"},
    {"Item": "Cheese", "Quantity": 1, "Expiry Date": "2025-03-20"},
    {"Item": "Apples", "Quantity": 6, "Expiry Date": "2025-03-25"},
    {"Item": "Carrots", "Quantity": 5, "Expiry Date": "2025-04-01"}
]
# Create an empty fridge inventory DataFrame with the required columns
fridge_df = pd.DataFrame(sample_data)

# Define the file path
fridge_file_path = "fridge_inventory.csv"

# Save the fridge DataFrame as a CSV file
fridge_df.to_csv(fridge_file_path, index=False)

In [8]:
try:
    fridge_df = pd.read_csv("fridge_inventory.csv")
except FileNotFoundError:
    fridge_df = pd.DataFrame(columns=["Item", "Quantity", "Expiry Date"])

## Step 2: Simulate tools

Here we implement the functions for some of the actual tool responses. In a real-world scenario, these functions would interact with your some systems in our household (or company), typically working with a database.

In [9]:
def list_fridge_contents():
    """Return the current contents of the fridge."""
    fridge_df = pd.read_csv("fridge_inventory.csv")
    return fridge_df.to_dict(orient="records")    

def get_fridge_contents_as_str_list():
    """Return the current contents of the fridge."""
    fridge_df = pd.read_csv("fridge_inventory.csv")
    return(", ".join(fridge_df["Item"].tolist()))
    
def add_to_fridge(item, quantity=1, expiry_date=""):
    """Add an item to the fridge."""
    fridge_df = pd.read_csv("fridge_inventory.csv")
    new_entry = pd.DataFrame([[item, quantity, expiry_date]], columns=["Item", "Quantity", "Expiry Date"])
    fridge_df = pd.concat([fridge_df, new_entry], ignore_index=True)
    fridge_df.to_csv("fridge_inventory.csv", index=False)
    return f"{item} added to fridge."

def calculate_total_fridge_cost():
    """
    Calculate the total cost of all items in the fridge.
    Assumes the fridge inventory CSV has a 'Price' column.
    If the 'Price' is not given (i.e., it's NA or 0), the 
    typical price is obtained from the internet.
    """
    if "Price" not in fridge_df.columns:
        fridge_df["Price"] = None
        
    for index, row in fridge_df.iterrows():
        if pd.isna(row["Price"]) or row["Price"] == 0:
            price = get_ai_response(f"What is usually the price of a single: {row['Item']}, reply only with a singular numeric value, no currency symbols.")
            fridge_df.at[index, "Price"] = price if price else "Unknown"

        if pd.isna(row["Quantity"]) or row["Quantity"] == 0:
            fridge_df.at[index, "Quantity"] = 1  # Default to 1 if missing

    # Convert price and quantity to numeric values (handle errors)
    fridge_df["Price"] = pd.to_numeric(fridge_df["Price"], errors="coerce")
    fridge_df["Quantity"] = pd.to_numeric(fridge_df["Quantity"], errors="coerce")

    # Calculate total cost
    total_cost = (fridge_df["Price"] * fridge_df["Quantity"]).sum()

    return {"total_cost": total_cost}

## Step 3: Process tool calls and return results

We'll create a function to process the various actions as identified by the household assistant LLM.

We are defining (simulating) several types of tools:
* Tools handling local data, such as `list_fridge_contents()` or `add_to_fridge`.
* Tools requiring information from a web search, such as `find_transport` from a given location to a given other location. Note that the LLM will also extract the locations based on the input query.

In [10]:
def assistant_call(assistant_action):
    print(assistant_action)
    if assistant_action['name'] == "get_weather":
        query = f"current weather in {assistant_action['params']['location']}"
        result = get_search_result(query)

        # Needed, as otherwise the token size for queries is too large
        fields_to_keep = ["temperature", 'unit', 'precipitation', 'humidity', 'wind', 'location', 'date', 'weather']
        result = {key: result[key] for key in fields_to_keep if key in result}

        ai_answer = get_ai_response(f"Based on this weather report give me a summary: {result}")
        return ai_answer
    
    elif assistant_action['name'] == "find_transport":
        query = f"public transport from {assistant_action['params']['from_location']} to {assistant_action['params']['to_location']}"
        result = get_search_result(query)
        ai_answer = get_ai_response(f'Summarize me these possible iteneraries {result}')
        return ai_answer
    
    elif assistant_action['name'] == "list_fridge_contents":
        contents = list_fridge_contents()
        ai_answer = get_ai_response(f"Based on this dictionary, what do I have in my fridge? {contents}, don't mention the dictionary in the reply")
        return ai_answer
    
    elif assistant_action['name'] == "add_to_fridge":
        return add_to_fridge(assistant_action['params']["item"], assistant_action['params']['quantity'], 
                             assistant_action['params']['expiry_date']) 
    
    elif assistant_action['name'] == "calculate_total_fridge_cost":
        return calculate_total_fridge_cost()
    
    else:
        return f"Error: Tool '{assistant_action['name']}' not recognized."

## Step 4: Describe the client-side tools

Next, we'll describe the tools (functions) that our household assistant will use to fulfil the tasks we tell it. The assistant will use this information to decide which tool to use for a given query.

For every one of these tools, we give the name of the function and the parameter(s) that the function takes (name and type of the value). Note the format (actually, the information is given in the JSON data interchange format). We choose file names that already contain a key description of the functionality.

In [11]:
actions = [
    {
        "name": "get_weather",
        "params": {
            "location": "string" # The queried location
        }
    },
    {
        "name": "find_transport",
        "params": {
            "from_location": "string", # From this location
            "to_location": "string", # To this location
        }
    },
    {
        "name": "get_search_result",
        "params" : {
            "query" : "string" # Query to be searched
        }
    },
    {
        "name": "list_fridge_contents",
        "params" : {}
    },
    {
        "name": "add_to_fridge",
        "params": {
            "item": "string",
            "quantity" : "integer",
            "expiry_date": "string",
         },
    },
    {
        "name": "calculate_total_fridge_cost",
        "params": {}
    }
]

## Step 5: Interact with the chatbot

Now, let's create a function to interact with the chatbot. We'll send a user message, use the LLM to find the right tool and extract the required parameters, and return the final response to the user.

In [12]:
import json
def household_assistant(user_message):
    prompt = f"Based on this user message: {user_message}, give me a matching action based on this action mapping:{actions}. Give me only the json as a string, no wrapping symbols."
    response = get_ai_response(prompt)
    response = json.loads(response)
    response = assistant_call(response)
    return response

## Step 6: Test the chatbot
Let's test our customer service chatbot with a few sample queries.

In [13]:
print(household_assistant("What's the weather right now in San Francisco, CA?"))

{'name': 'get_weather', 'params': {'location': 'San Francisco, CA'}}
The weather in San Francisco, CA on Friday at 12:00 PM is sunny with a temperature of 59°F, 0% chance of precipitation, 50% humidity, and a 5 mph wind.


In [14]:
print(household_assistant("Find a recipe from the items in my fridge"))

{'name': 'list_fridge_contents', 'params': {}}
In your fridge, you have milk, eggs, cheese, apples, and carrots.


In [15]:
print(household_assistant("How can I reach Seebacherplatz from Hohlstrasse?"))


{'name': 'find_transport', 'params': {'from_location': 'Hohlstrasse', 'to_location': 'Seebacherplatz'}}
Here are some possible itineraries for your journey from Hohlstrasse, Zürich, Switzerland to Seebacherplatz, 8052 Zürich, Switzerland:

1. Route 1: Departure at 10:26 PM, arrival at 11:01 PM, duration 36 min. Departure from SBB Werkstätte at 10:28 PM.
2. Route 2: Departure at 10:20 PM, arrival at 10:53 PM, duration 33 min. Departure from Hardbrücke at 10:39 PM.
3. Route 3: Departure at 10:36 PM, arrival at 11:11 PM, duration 36 min. Departure from SBB Werkstätte at 10:38 PM.
4. Route 4: Departure at 10:26 PM, arrival at 11:01 PM, duration 36 min. Departure from Hardplatz at 10:38 PM.
5. Route 5: Departure at 10:20 PM, arrival at 10:58 PM, duration 38 min. Departure from Hardbrücke at 10:39 PM.
6. Route 6: Departure at 10:46 PM, arrival at 11:21 PM, duration 36 min. Departure from SBB Werkstätte at 10:48 PM.


In [16]:
print(household_assistant("What is in my fridge?"))

{'name': 'list_fridge_contents', 'params': {}}
In your fridge, you have milk, eggs, cheese, apples, and carrots.


In [17]:
print(household_assistant("I bought two twinkies that expire on the 2025-05-01, I've added them to my fridge."))

{'name': 'add_to_fridge', 'params': {'item': 'twinkies', 'quantity': 2, 'expiry_date': '2025-05-01'}}
twinkies added to fridge.


In [18]:
print(household_assistant("What's in my fridge?"))

{'name': 'list_fridge_contents', 'params': {}}
In your fridge, you have milk, eggs, cheese, apples, carrots, and twinkies.


In [19]:
print(household_assistant("What is the total value of the items in my fridge?"))

{'name': 'calculate_total_fridge_cost', 'params': {}}
{'total_cost': 51}


Note that the LLM will have to choose one of the tools you made available. For example, there no tool (yet) to remove something from the fridge. Instead, the LLM will provide a list of the current fridge items:

In [20]:
print(household_assistant("Please remove the carrots from my fridge, those are way too unhealthy"))

{'name': 'list_fridge_contents', 'params': {}}
In your fridge, you have milk, eggs, cheese, apples, carrots, and twinkies.


## Extend the chatbot
**Exercise:** Add some more tools to the chatbot. If you need inspiration, what about:
- remove items from the fridge inventory list
- adding / removing items to a shopping list
- find concerts that will take place in your city in the next two weeks
- find a few recipes for the items currently in the fridge
    - remember that there's already a function that returns the items currently in your fridge. Use it to create a search query.

Remeber to add your new tool 
* in the `actions` map, so that the assistant is aware of the action and can identify if it's the correct tool for a given request.
* in the `assistant_call` function, so that the assistant can actually call that tool (function) when it thinks it's the right tool for the request.
* possibly some other function (such as `list_fridge_contents` or `get_fridge_contents_as_str_list`) to describe the functionality that is needed.